In [1]:
# prompt: googlemount

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==24.8.*




Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.7/516.7 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.8/783.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 102.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.0/916.0 kB 4

In [ ]:
    # cugraph-cu12==24.8.* \
    # cuspatial-cu12==24.8.* \
    # cuproj-cu12==24.8.* \
    # cuxfilter-cu12==24.8.* \
    # cucim-cu12==24.8.* \
    # pylibraft-cu12==24.8.* \
    # raft-dask-cu12==24.8.* \
    # cudf-cu12==24.8.* \
    # dask-cudf-cu12==24.8.* \
    # cuvs-cu12==24.8.*

In [5]:
import cuml
cuml.__version__

'24.08.00'

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import sklearn
import math
import statistics
import warnings
import os

warnings.filterwarnings(action='ignore')
pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_columns',None)

In [7]:
temp_data=pd.read_csv(r"/content/drive/MyDrive/lending_club/Cleansing_data1_out_current_order_by_date.csv",low_memory=False,keep_default_na=True)

In [8]:
temp_data['target'] = temp_data['total_rec_prncp']/temp_data['funded_amnt']

In [ ]:
# temp_data = temp_data.dropna(subset=['target']) # 결측치행 제거

In [9]:
temp_data['target'].isna().sum()

0

In [14]:
# temp_data에서 랜덤으로 10000개 행 추출
temp_data = temp_data.sample(n=10000, random_state=42)

# 결과 확인
len(temp_data)

10000

In [15]:
import pandas as pd
import numpy as np
from cuml.ensemble import RandomForestRegressor
from cuml.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [16]:
X = temp_data.drop('target', axis=1)
y = temp_data['target']



In [17]:
# Assume temp_data is defined previously with appropriate data
# X = temp_data.drop('target', axis=1)
# y = temp_data['target']

# 스케일링은 적당히 조정
def stsc(x):
    stdsc = StandardScaler()
    temp_x = x.select_dtypes(exclude='object')
    name = temp_x.columns
    stdsc.fit(np.array(temp_x).reshape(-1, len(name)))
    temp_x = stdsc.transform(np.array(temp_x).reshape(-1, len(name)))
    temp_x = pd.DataFrame(temp_x)
    temp_x.columns = name
    return temp_x

# Scaling the data
temp_x = stsc(X)

# Step 1: 먼저 80%를 train+validation 세트로, 20%를 test 세트로 나눕니다.
x_train_val, x_test, y_train_val, y_test = train_test_split(temp_x, y, test_size=0.2, random_state=42)

# Step 2: train+validation 세트를 다시 6:2 비율로 train 세트와 validation 세트로 나눕니다.
x_train, x_valid, y_train, y_valid = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=42)

# 넘파이 배열로 바꿔주기
x_train_np = x_train.to_numpy()
x_test_np = x_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()
x_valid_np = x_valid.to_numpy()
y_valid_np = y_valid.to_numpy()

# 랜덤 포레스트 모델 설정
cuml_rf = RandomForestRegressor()

# 그리드 서치 파라미터 설정
param_grid = {
    'n_estimators': [50, 100, 200],  # 트리의 개수
    'max_depth': [5, 10, 20],        # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10]  # 노드를 나누기 위한 최소 샘플 수
}

# 그리드 서치 모델 설정
cuml_grid_search = GridSearchCV(estimator=cuml_rf, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error', verbose=1)

# 모델 적합
cuml_grid_search.fit(x_train_np, y_train_np)

# 평가
print(cuml_grid_search.best_params_)
print(cuml_grid_search.best_score_)

# Validation 세트에 대한 예측
y_valid_pred = cuml_grid_search.predict(x_valid_np)

# Validation 세트에 대한 평가
validation_rmse = np.sqrt(mean_squared_error(y_valid_np, y_valid_pred))
print(f"Validation RMSE: {validation_rmse:.4f}")

Fitting 5 folds for each of 27 candidates, totalling 135 fits
{'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}
-0.05233841515398942
Validation RMSE: 0.0510
